In [ ]:
pip install pandas sqlalchemy


In [ ]:
pip install mysql-connector-python


In [ ]:
import pandas as pd
from sqlalchemy import create_engine, Column, String, DateTime, Float, Integer, Boolean, BigInteger
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy.exc import IntegrityError

# Ruta al archivo Excel (xlsx)
excel_path = input("Por favor, ingrese la ruta del archivo Excel: ")

# Configuración de la conexión a la base de datos MySQL
db_config = {
    'host': 'localhost',
    'user': 'root',
    'port': 3306,
    'password': '',
    'database': 'pensiones'
}

# Crear una conexión a la base de datos MySQL
engine = create_engine(f"mysql+mysqlconnector://{db_config['user']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['database']}")

Base = declarative_base()

# Define la estructura de la tabla
class Datos(Base):
    __tablename__ = 'datos'
    Nombre = Column(String(collation='utf8mb4_general_ci'))
    Apellidos = Column(String(collation='utf8mb4_general_ci'))
    Numero_Identificacion = Column(String(100), primary_key=True)
    Fecha_de_Nacimiento = Column(DateTime)
    Fecha_de_Jubilacion = Column(DateTime)
    Bases_de_Cotizacion = Column(Float)
    Comunidad_Autonoma = Column(String(collation='utf8mb4_general_ci'))
    Invalidez = Column(Boolean)
    Base_Reguladora = Column(Float)
    Total_Meses_Cotizados = Column(Integer)
    Pension = Column(Float)
    Pension_Final = Column(Float)
    Anos_Enteros_Cotizados = Column(Integer)
    Meses_Cotizados = Column(Integer)
    Anos_Cotizados_Decimal = Column(Float)
    Edad = Column(Integer)
    Grupo_Edades = Column(String(collation='utf8mb4_general_ci'))
    Pension_Final_con_Complementos = Column(Float)

# Crear la tabla si no existe
Base.metadata.create_all(engine)

# Crear una sesión de SQLAlchemy
Session = sessionmaker(bind=engine)
session = Session()

# Leer el archivo Excel en un DataFrame
df = pd.read_excel(excel_path)

# DataFrame para registros erróneos
registros_erroneos = pd.DataFrame(columns=df.columns)

# Itera sobre las filas del DataFrame y agrega o actualiza los datos en la base de datos
for index, row in df.iterrows():
    # Comprobar si alguno de los campos está vacío
    if row.isnull().any():
        registros_erroneos = registros_erroneos.append(row)
    else:
        try:
            dato = Datos(
                Nombre=row['Nombre'],
                Apellidos=row['Apellidos'],
                Numero_Identificacion=str(row['Numero_Identificacion']),
                Fecha_de_Nacimiento=row['Fecha_de_Nacimiento'],
                Fecha_de_Jubilacion=row['Fecha_de_Jubilacion'],
                Bases_de_Cotizacion=row['Bases_de_Cotizacion'],
                Comunidad_Autonoma=row['Comunidad_Autonoma'],
                Invalidez=bool(row['Invalidez']),
                Base_Reguladora=row['Base_Reguladora'],
                Total_Meses_Cotizados=row['Total_Meses_Cotizados'],
                Pension=row['Pension'],
                Pension_Final=row['Pension_Final'],
                Anos_Enteros_Cotizados=row['Anos_Enteros_Cotizados'],
                Meses_Cotizados=row['Meses_Cotizados'],
                Anos_Cotizados_Decimal=row['Anos_Cotizados_Decimal'],
                Edad=row['Edad'],
                Grupo_Edades=row['Grupo_Edades'],
                Pension_Final_con_Complementos=row['Pension_Final_con_Complementos']
            )
            session.merge(dato)
        except Exception as e:
            registros_erroneos = registros_erroneos.append(row)

# Intenta guardar los cambios en la base de datos
try:
    session.commit()
    print('Datos válidos cargados en la tabla "datos".')
except IntegrityError as e:
    session.rollback()
    print('Error al cargar los datos válidos:', str(e))

# Guarda los registros erróneos en un archivo Excel
if not registros_erroneos.empty:
    registros_erroneos.to_excel('Registros_Erroneos.xlsx', index=False)
    print('Registros erróneos guardados en "Registros_Erroneos.xlsx".')
